In [1]:
# data manipulation modules
import pandas as pd
import numpy as np

# general python modules
import json
import csv
import io 
import os

# dialogflow cx imports
from dfcx_scrapi.core.agents import Agents
from dfcx_scrapi.core.intents import Intents
from dfcx_scrapi.core.flows import Flows
from dfcx_scrapi.core.pages import Pages
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions
from dfcx_scrapi.tools.maker_util import MakerUtil

# For connect to google sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

## -- Uncomment the below section if you want to auth with Google Drive -- ##
# from google.colab import auth
# from google.auth import default

# # Mount your Google Drive.
# from google.colab import drive
# drive.mount("/content/drive"

In [2]:
# service account scopes
# master systems analysis and design dxfc g-sheets workbook... Can be viewed and copied at https://itrau.co/dfcx-sad

scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']


#authorize sa
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/christophertrauco/.config/secrets/cx_admin_sa.json', scope)
client = gspread.authorize(credentials)
spreadsheet_key = '1N10vIxxT9nJShDDNbFvIAxcCOe8vLhoZ_8Ex3lsGc_c'
sh = client.open_by_key(spreadsheet_key)
project_id = sh.worksheet('Main').acell('B1')
print(project_id)
# master systems analysis and design dxfc g-sheets workbook... Can be viewed and copied at https://itrau.co/dfcx-sad


<Cell R1C2 'bot1-404120'>


In [3]:
creds_path = sh.worksheet('Main').acell('B8').value
project_id = sh.worksheet('Main').acell('B1').value
agent_display_name = "bc-records-agent"
gcp_region = sh.worksheet('Main').acell('B3').value

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = creds_path

In [4]:
# Instantiate the core classes
agents_instance = Agents(creds_path=creds_path)
flows_instance = Flows(creds_path=creds_path)
intents_instance = Intents(creds_path=creds_path)
pages_instance = Pages(creds_path=creds_path)
entity_types_instance = EntityTypes(creds_path=creds_path)
webhooks_instance = Webhooks(creds_path=creds_path)
environments_instance = Environments(creds_path=creds_path)
route_groups_instance = TransitionRouteGroups(creds_path=creds_path)

# Instantiate the tools classes
dffx_instance = DataframeFunctions(creds_path=creds_path)

# Get the Agent and its ID
my_agent = agents_instance.get_agent_by_display_name(
    project_id=project_id, display_name=agent_display_name
)

agent_id = my_agent.name

NameError: name 'EntityTypes' is not defined

In [ ]:
# Load the flows
all_flows = flows_instance.list_flows(agent_id=agent_id)

flows_map = flows_instance.get_flows_map(agent_id=agent_id)
flows_map_reverse = flows_instance.get_flows_map(
    agent_id=agent_id, reverse=True
)

# Load the pages
all_pages_per_flow = {}

pages_map_per_flow = {}
pages_map_per_flow_reverse = {}

for flow_name, flow_id in flows_map_reverse.items():
    all_pages_per_flow[flow_name] = pages_instance.list_pages(flow_id=flow_id)
    time.sleep(1)
    
    pages_map_per_flow[flow_name] = pages_instance.get_pages_map(
        flow_id=flow_id
    )
    time.sleep(1)
    pages_map_per_flow_reverse[flow_name] = pages_instance.get_pages_map(
        flow_id=flow_id, reverse=True
    )

In [ ]:
# Load the intents
all_intents = intents_instance.list_intents(agent_id=agent_id)

intents_map = intents_instance.get_intents_map(agent_id=agent_id)
intents_map_reverse = intents_instance.get_intents_map(
    agent_id=agent_id, reverse=True
)

# Load the entity types
all_entity_types = entity_types_instance.list_entity_types(agent_id=agent_id)

entity_types_map = entity_types_instance.get_entities_map(agent_id=agent_id)
entity_types_map_reverse = entity_types_instance.get_entities_map(
    agent_id=agent_id, reverse=True
)

In [ ]:
# Load the webhooks
all_webhooks = webhooks_instance.list_webhooks(agent_id=agent_id)

webhooks_map = webhooks_instance.get_webhooks_map(agent_id=agent_id)
webhooks_map_reverse = webhooks_instance.get_webhooks_map(
    agent_id=agent_id, reverse=True
)

# Load the environments
all_environments = environments_instance.list_environments(agent_id=agent_id)

environments_map = environments_instance.get_environments_map(agent_id=agent_id)
environments_map_reverse = environments_instance.get_environments_map(
    agent_id=agent_id, reverse=True
)

# Load the transition route groups
all_route_groups_per_flow = {}

route_groups_map_per_flow = {}
route_groups_map_per_flow_reverse = {}

for flow_name, flow_id in flows_map_reverse.items():
    route_groups_list = route_groups_instance.list_transition_route_groups(
        flow_id=flow_id
    )
    all_route_groups_per_flow[flow_name] = route_groups_list
    time.sleep(1)
    
    route_groups_map = route_groups_instance.get_route_groups_map(
        flow_id=flow_id
    )
    route_groups_map_per_flow[flow_name] = route_groups_map
    time.sleep(1)
    route_groups_map_reverse = route_groups_instance.get_route_groups_map(
        flow_id=flow_id, reverse=True
    )
    route_groups_map_per_flow_reverse[flow_name] = route_groups_map_reverse

In [ ]:
from dfcx_scrapi.tools.copy_util import CopyUtil

cu = CopyUtil(creds_path)

flows_map = cu.flows.get_flows_map(agent_id, reverse=True)

In [ ]:
print(flows_map)

In [ ]:
df = pd.DataFrame(flows_map,index=[1])
df.reset_index(drop=True)
# result = {} # empty dictionary
# for li in df:
#     result.setdefault(li[0],[]).append(li[1])
# print(result)
print(df)
# print(entity_types_map)
# type(entity_types_map)

wks_name = 'Flows'
cell_of_start_df = 'A1'
# upload the dataframe of the clients we want to delete
d2g.upload(df,
           spreadsheet_key,
           wks_name,
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = cell_of_start_df,
           clean=True)
print ('The sheet is updated successfully')

In [ ]:
source_flow = 'Default Start Flow'
target_flow = 'My Order Flow'



my_pages = cu.pages.list_pages(flows_map[source_flow])
print('{} Page Count = {}'.format(source_flow, len(my_pages)))
print(my_pages)

In [ ]:
create_new_proto_obj("test")

In [ ]:
class FlowBuilder(BuildersCommon):
    """Base Class for CX Flow builder."""

    _proto_type = Flow
    _proto_type_str = "Flow"

    
    def __str__(self) -> str:
        """String representation of the proto_obj."""
        self._check_proto_obj_attr_exist()

        return (
            f"Basic Information:\n{'='*25}\n{self._show_basic_info()}"
            f"\n\n\nTransitionRoutes:\n{'='*25}"
            f"\n{self._show_transition_routes()}"
            f"\n\n\nEventHandlers:\n{'='*25}\n{self._show_event_handlers()}"
            f"\n\n\nTransitoinRouteGroups:\n{'='*25}"
            f"\n{self._show_transition_route_groups()}")


    def _show_basic_info(self) -> str:
        """String representation for the basic information of proto_obj."""
        self._check_proto_obj_attr_exist()

        nlu_settings_str = (
            f"\tModel type: {self.proto_obj.nlu_settings.model_type.name}"
            "\n\tClassification threshold:"
            f" {self.proto_obj.nlu_settings.classification_threshold}"
            "\n\tTraining mode:"
            f" {self.proto_obj.nlu_settings.model_training_mode.name}"
        )

        return (
            f"display_name: {self.proto_obj.display_name}"
            f"\ndescription:\n\t{self.proto_obj.description}"
            f"\nNLU settings:\n{nlu_settings_str}"
        )


In [5]:
    def show_flow_info(
        self, mode: str = "whole"
    ) -> None:
        """Show the proto_obj information.

        Args:
          mode (str):
            Specifies what part of the page to show.
              Options:
              ['basic', 'whole',
              'routes' or 'transition routes',
              'route groups' or 'transition route groups',
              'events' or 'event handlers'
              ]
        """
        self._check_proto_obj_attr_exist()

        if mode == "basic":
            print(self._show_basic_info())
        elif mode in ["routes", "transition routes"]:
            print(self._show_transition_routes())
        elif mode in ["route groups", "transition route groups"]:
            print(self._show_transition_route_groups())
        elif mode in ["events", "event handlers"]:
            print(self._show_event_handlers())
        elif mode == "whole":
            print(self)
        else:
            raise ValueError(
                "mode should be in"
                "['basic', 'whole',"
                " 'routes', 'transition routes',"
                " 'route groups', 'transition route groups',"
                " 'events', 'event handlers']"
            )


In [7]:
show_flow_info('basic')

AttributeError: 'str' object has no attribute '_check_proto_obj_attr_exist'

In [11]:
"""CX Transition Route Group Resource functions."""

# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import logging
import time
from typing import Dict
import pandas as pd
from google.cloud.dialogflowcx_v3beta1 import services
from google.cloud.dialogflowcx_v3beta1 import types
from google.protobuf import field_mask_pb2

from dfcx_scrapi.core import flows
from dfcx_scrapi.core import intents
from dfcx_scrapi.core import pages
from dfcx_scrapi.core import webhooks
from dfcx_scrapi.core import scrapi_base


# logging config
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)


class TransitionRouteGroups(scrapi_base.ScrapiBase):
    """Core Class for CX Transition Route Group functions."""

    def __init__(
        self,
        creds_path: str = None,
        creds_dict: Dict = None,
        creds=None,
        scope=False,
        route_group_id: str = None,
        flow_id: str = None,
        agent_id: str = None,
    ):
        super().__init__(
            creds_path=creds_path,
            creds_dict=creds_dict,
            creds=creds,
            scope=scope,
        )

        self.flows = flows.Flows(creds=self.creds)
        self.intents = intents.Intents(creds=self.creds)
        self.pages = pages.Pages(creds=self.creds)
        self.webhooks = webhooks.Webhooks(creds=self.creds)

        if route_group_id:
            self.route_group_id = route_group_id
            self.client_options = self._set_region(route_group_id)

        if flow_id:
            self.flow_id = flow_id

        if agent_id:
            self.agent_id = agent_id

    def _rg_temp_dict_update(self, temp_dict, element):
        """Modify the temp dict and return to dataframe function."""
        element_dict = self.cx_object_to_dict(element)
        key = list(element_dict.keys())[0]

        if key == "payload":
            temp_dict.update({"custom_payload": element_dict[key]})
        elif key == "liveAgentHandoff":
            temp_dict.update(
                {"live_agent_handoff": element_dict[key]["metadata"]}
            )
        elif key == "conversationSuccess":
            temp_dict.update(
                {"conversation_success": element_dict[key]["metadata"]}
            )
        elif key == "playAudio":
            temp_dict.update({"play_audio": element_dict[key]["audioUri"]})
        elif key == "outputAudioText":
            temp_dict.update({"output_audio_text": element_dict[key]["text"]})
        elif key == "text":
            if len(element_dict[key]["text"]) == 1:
                temp_dict.update(
                    {"fulfillment_message": element_dict[key]["text"][0]}
                )
            else:
                temp_dict.update(
                    {"fulfillment_message": element_dict[key]["text"]}
                )
        else:
            temp_dict.update({key: element_dict[key]})

        return temp_dict

    def get_route_groups_map(self, flow_id: str = None, reverse=False):
        """Exports Agent Route Group UUIDs and Names into a user friendly dict.

        Args:
          flow_id: the formatted CX Agent Flow ID to use
          reverse: (Optional) Boolean flag to swap key:value -> value:key

        Returns:
          Dictionary containing Route Group UUIDs as keys and display names
          as values. If Optional reverse=True, the output will return
          route group name:ID mapping instead of ID:route group name
        """
        if not flow_id:
            flow_id = self.flow_id

        if reverse:
            pages_dict = {
                page.display_name: page.name
                for page in self.list_transition_route_groups(flow_id)
            }

        else:
            pages_dict = {
                page.name: page.display_name
                for page in self.list_transition_route_groups(flow_id)
            }

        return pages_dict

    @scrapi_base.api_call_counter_decorator
    def list_transition_route_groups(self, flow_id: str = None):
        """Exports List of all Route Groups in the specified CX Flow ID.

        Args:
          flow_id: The formatted CX Flow ID to list the route groups from

        Returns:
          List of Route Group objects
        """
        if not flow_id:
            flow_id = self.flow_id

        request = (
            types.transition_route_group.ListTransitionRouteGroupsRequest()
        )
        request.parent = flow_id

        client_options = self._set_region(flow_id)
        client = services.transition_route_groups.TransitionRouteGroupsClient(
            credentials=self.creds, client_options=client_options
        )
        response = client.list_transition_route_groups(request)

        cx_route_groups = []
        for page in response.pages:
            for cx_route_group in page.transition_route_groups:
                cx_route_groups.append(cx_route_group)

        return cx_route_groups

    @scrapi_base.api_call_counter_decorator
    def get_transition_route_group(self, route_group_id):
        """Get a single Transition Route Group object.

        Args:
          route_group_id: the formatted CX Route Group ID to retrieve.

        Returns:
          A single Route Group object
        """
        request = types.transition_route_group.GetTransitionRouteGroupRequest()
        request.name = route_group_id
        client_options = self._set_region(route_group_id)
        client = services.transition_route_groups.TransitionRouteGroupsClient(
            credentials=self.creds, client_options=client_options
        )
        response = client.get_transition_route_group(request)

        return response

    @scrapi_base.api_call_counter_decorator
    def create_transition_route_group(
        self,
        flow_id: str = None,
        obj: types.TransitionRouteGroup = None,
        **kwargs,
    ):
        """Create a single Transition Route Group resource.

        Args:
          flow_id: the formatted CX Flow ID to create the route group in
          obj: (Optional) the Transition Route Group object of type
            types.TransitionRouteGroup that you want the new route group
            to be built from.

        Returns:
          A copy of the successfully created Route Group object
        """
        if not flow_id:
            flow_id = self.flow_id

        if obj:
            trg = obj
            trg.name = ""
        else:
            trg = types.transition_route_group.TransitionRouteGroup()

        for key, value in kwargs.items():
            setattr(trg, key, value)

        client_options = self._set_region(flow_id)
        client = services.transition_route_groups.TransitionRouteGroupsClient(
            credentials=self.creds, client_options=client_options
        )
        response = client.create_transition_route_group(
            parent=flow_id, transition_route_group=trg
        )

        return response

    @scrapi_base.api_call_counter_decorator
    def update_transition_route_group(
        self,
        route_group_id: str = None,
        obj: types.TransitionRouteGroup = None,
        language_code: str = None,
        **kwargs,
    ):
        """Update a single Route Group resource.

        Args:
          route_group_id: the formatted CX Route Group ID to update.
          obj: (Optional) the Transition Route Group object of type
            types.TransitionRouteGroup that you want to update.
          language_code: (Optional) the language in which the agent should
            update the TransitionRouteGroup

        Returns:
          A copy of the successfully updated Route Group object
        """
        if obj:
            route_group = obj
            route_group.name = route_group_id
        else:
            route_group = self.get_transition_route_group(route_group_id)

        for key, value in kwargs.items():
            setattr(route_group, key, value)
        paths = kwargs.keys()
        mask = field_mask_pb2.FieldMask(paths=paths)

        client_options = self._set_region(route_group_id)
        client = services.transition_route_groups.TransitionRouteGroupsClient(
            credentials=self.creds, client_options=client_options
        )

        request = (
            types.transition_route_group.UpdateTransitionRouteGroupRequest()
        )  # pylint: disable=C0301

        request.transition_route_group = route_group
        request.update_mask = mask

        if language_code:
            request.language_code = language_code

        response = client.update_transition_route_group(request)

        return response

    def route_groups_to_dataframe(
        self, agent_id: str = None, rate_limit: float = 0.5
    ):
        """Extracts the Transition Route Groups from a given Agent and
         returns key information about the Route Groups in a Pandas Dataframe

        DFCX Route Groups exist as an Agent level resource, however they are
        categorized by the Flow they are associated with. This method will
        extract all Flows for the given agent, then use the Flow IDs to
        extract all Route Groups per Flow. Once all Route Groups have been
        extracted, the method will convert the DFCX object to a Pandas
        Dataframe and return this to the user.

        Args:
          agent_id: the Agent ID string in the following format:
            projects/<project_id>/locations/<location_id>/agents/<agent_id>
          rate_limit: Time in seconds to wait between each API call. Use this
            to control hitting Quota limits on your project.

        Returns:
          a Pandas Dataframe with columns: flow, route_group_name, target_page,
          intent, condition, webhook, webhook_tag, custom_payload,
          live_agent_handoff, conversation_success, play_audio,
          output_audio_text, fulfillment_message
        """
        if not agent_id:
            agent_id = self.agent_id

        flows_map = self.flows.get_flows_map(agent_id)
        intents_map = self.intents.get_intents_map(agent_id)
        webhooks_map = self.webhooks.get_webhooks_map(agent_id)
        all_pages_map = {}
        all_rgs = []

        for flow in flows_map:
            all_pages_map.update(self.pages.get_pages_map(flow))
            all_rgs.extend(self.list_transition_route_groups(flow))
            time.sleep(rate_limit)

        rows_list = []
        for route_group in all_rgs:
            flow = "/".join(route_group.name.split("/")[0:8])
            for route in route_group.transition_routes:
                temp_dict = {}

                temp_dict.update({"flow": flows_map[flow]})
                temp_dict.update({"route_group_name": route_group.display_name})

                if route.target_page:
                    t_p = all_pages_map.get(route.target_page)
                    if not t_p:
                        t_p = str(route.target_page).rsplit("/", maxsplit=1)[-1]

                    temp_dict.update({"target_page": t_p})

                if route.intent:
                    temp_dict.update({"intent": intents_map[route.intent]})

                if route.condition:
                    temp_dict.update({"condition": route.condition})

                if route.trigger_fulfillment.webhook:
                    temp_dict.update(
                        {
                            "webhook": webhooks_map[
                                route.trigger_fulfillment.webhook
                            ]
                        }
                    )

                    temp_dict.update(
                        {"webhook_tag": route.trigger_fulfillment.tag}
                    )

                if route.trigger_fulfillment.messages:
                    for element in route.trigger_fulfillment.messages:
                        temp_dict = self._rg_temp_dict_update(
                            temp_dict, element
                        )

                rows_list.append(temp_dict)

        final_dataframe = pd.DataFrame(rows_list)

        return final_dataframe

In [10]:
create_transition_route_group_str()

NameError: name 'create_transition_route_group_str' is not defined

In [19]:
project_id = sh.worksheet('Main').acell('B1').value
display_name = sh.worksheet('Main').acell('B2').value
gcp_region = sh.worksheet('Main').acell('B3').value
zone = sh.worksheet('Main').acell('B4').value
language = sh.worksheet('Main').acell('B5').value
location = 'global'
time_zone = sh.worksheet('Main').acell('B6').value
creds_path = sh.worksheet('Main').acell('B8').value

In [32]:
from google.cloud import dialogflowcx_v3


def sample_create_flow():
    # Create a client
    client = dialogflowcx_v3.FlowsClient()

    # Initialize request argument(s)
    flow = dialogflowcx_v3.Flow()
    flow.display_name = "test"

    request = dialogflowcx_v3.CreateFlowRequest(
        parent="projects/bot1-404120/locations/global/agents/a8b82c5c-9170-4604-8a4c-362a0004a97f",
        flow=flow,
    )

    # Make the request
    response = client.create_flow(request=request)

    # Handle the response
    print(response)

# [END dialogflow_v3_generated_Flows_CreateFlow_sync]

In [28]:
sample_create_flow()

InvalidArgument: 400 Resource name 'parent_value' does not match 'projects/*/locations/*/agents/*'.